In [1]:
from __future__ import print_function
import os
from os.path import splitext, join, isfile, isdir, basename
import argparse
import numpy as np
from scipy import misc, ndimage
from keras import backend as K
from keras.models import model_from_json, load_model
import tensorflow as tf
import layers_builder as layers
from glob import glob
from utils import utils
from keras.utils.generic_utils import CustomObjectScope
import cv2
import math
# -- Fix for macos, uncomment it
# import matplotlib
# matplotlib.use('TkAgg')
# --
import matplotlib.pyplot as plt
import cvlib as cv
from cvlib.object_detection import draw_bbox
from sklearn.cluster import KMeans
from collections import Counter

from imageio import imread
import json
import time
# These are the means for the ImageNet pretrained ResNet
DATA_MEAN = np.array([[[123.68, 116.779, 103.939]]])  # RGB order

Using TensorFlow backend.


In [2]:
class PSPNet(object):
    """Pyramid Scene Parsing Network by Hengshuang Zhao et al 2017"""

    def __init__(self, nb_classes, resnet_layers, input_shape, weights):
        self.input_shape = input_shape
        self.num_classes = nb_classes

        json_path = join("weights", "keras", weights + ".json")
        h5_path = join("weights", "keras", weights + ".h5")
        if 'pspnet' in weights:
            if os.path.isfile(json_path) and os.path.isfile(h5_path):
                print("Keras model & weights found, loading...")
                with CustomObjectScope({'Interp': layers.Interp}):
                    with open(json_path) as file_handle:
                        self.model = model_from_json(file_handle.read())
                self.model.load_weights(h5_path)
            else:
                print("No Keras model & weights found, import from npy weights.")
                self.model = layers.build_pspnet(nb_classes=nb_classes,
                                                 resnet_layers=resnet_layers,
                                                 input_shape=self.input_shape)
                self.set_npy_weights(weights)
        else:
            print('Load pre-trained weights')
            self.model = load_model(weights)

    def predict(self, img, flip_evaluation=False):
        """
        Predict segementation for an image.
        Arguments:
            img: must be rowsxcolsx3
        """

        if img.shape[0:2] != self.input_shape:
            print(
                "Input %s not fitting for network size %s, resizing. You may want to try sliding prediction for better results." % (
                img.shape[0:2], self.input_shape))
            img = misc.imresize(img, self.input_shape)

        img = img - DATA_MEAN
        img = img[:, :, ::-1]  # RGB => BGR
        img = img.astype('float32')

        probs = self.feed_forward(img, flip_evaluation)

        return probs

    def predict_sliding(self, full_img, flip_evaluation):
        """
        Predict on tiles of exactly the network input shape.
        This way nothing gets squeezed.
        """
        tile_size = self.input_shape
        classes = self.num_classes
        overlap = 1 / 3

        stride = math.ceil(tile_size[0] * (1 - overlap))
        tile_rows = max(int(math.ceil((full_img.shape[0] - tile_size[0]) / stride) + 1), 1)  # strided convolution formula
        tile_cols = max(int(math.ceil((full_img.shape[1] - tile_size[1]) / stride) + 1), 1)
        print("Need %i x %i prediction tiles @ stride %i px" % (tile_cols, tile_rows, stride))
        full_probs = np.zeros((full_img.shape[0], full_img.shape[1], classes))
        count_predictions = np.zeros((full_img.shape[0], full_img.shape[1], classes))
        tile_counter = 0
        for row in range(tile_rows):
            for col in range(tile_cols):
                x1 = int(col * stride)
                y1 = int(row * stride)
                x2 = min(x1 + tile_size[1], full_img.shape[1])
                y2 = min(y1 + tile_size[0], full_img.shape[0])
                x1 = max(int(x2 - tile_size[1]), 0)  # for portrait images the x1 underflows sometimes
                y1 = max(int(y2 - tile_size[0]), 0)  # for very few rows y1 underflows

                img = full_img[y1:y2, x1:x2]
                padded_img = self.pad_image(img, tile_size)
#                 plt.imshow(padded_img)
#                 plt.show()
                tile_counter += 1
                print("Predicting tile %i" % tile_counter)
                padded_prediction = self.predict(padded_img, flip_evaluation)
                prediction = padded_prediction[0:img.shape[0], 0:img.shape[1], :]
                count_predictions[y1:y2, x1:x2] += 1
                full_probs[y1:y2, x1:x2] += prediction  # accumulate the predictions also in the overlapping regions

        # average the predictions in the overlapping regions
        full_probs /= count_predictions
        # visualize normalization Weights
        # plt.imshow(np.mean(count_predictions, axis=2))
        # plt.show()
        return full_probs

    @staticmethod
    def pad_image(img, target_size):
        """Pad an image up to the target size."""
        rows_missing = target_size[0] - img.shape[0]
        cols_missing = target_size[1] - img.shape[1]
        padded_img = np.pad(img, ((0, rows_missing), (0, cols_missing), (0, 0)), 'constant')
        return padded_img

    def predict_multi_scale(self, img, flip_evaluation, sliding_evaluation, scales):
        """Predict an image by looking at it with different scales."""

        full_probs = np.zeros((img.shape[0], img.shape[1], self.num_classes))
        h_ori, w_ori = img.shape[:2]

        print("Started prediction...")
        for scale in scales:
            print("Predicting image scaled by %f" % scale)
            scaled_img = misc.imresize(img, size=scale, interp="bilinear")

            if sliding_evaluation:
                scaled_probs = self.predict_sliding(scaled_img, flip_evaluation)
            else:
                scaled_probs = self.predict(scaled_img, flip_evaluation)

            # scale probs up to full size
            # visualize_prediction(probs)
            probs = cv2.resize(scaled_probs, (w_ori, h_ori))
            full_probs += probs
        full_probs /= len(scales)
        print("Finished prediction...")

        return full_probs

    def feed_forward(self, data, flip_evaluation=False):
        assert data.shape == (self.input_shape[0], self.input_shape[1], 3)

        if flip_evaluation:
            print("Predict flipped")
            input_with_flipped = np.array(
                [data, np.flip(data, axis=1)])
            prediction_with_flipped = self.model.predict(input_with_flipped)
            prediction = (prediction_with_flipped[
                          0] + np.fliplr(prediction_with_flipped[1])) / 2.0
        else:
            prediction = self.model.predict(np.expand_dims(data, 0))[0]
        return prediction

    def set_npy_weights(self, weights_path):
        npy_weights_path = join("weights", "npy", weights_path + ".npy")
        json_path = join("weights", "keras", weights_path + ".json")
        h5_path = join("weights", "keras", weights_path + ".h5")

        print("Importing weights from %s" % npy_weights_path)
        weights = np.load(npy_weights_path, encoding='bytes').item()
        for layer in self.model.layers:
            print(layer.name)
            if layer.name[:4] == 'conv' and layer.name[-2:] == 'bn':
                mean = weights[layer.name.encode()][
                    'mean'.encode()].reshape(-1)
                variance = weights[layer.name.encode()][
                    'variance'.encode()].reshape(-1)
                scale = weights[layer.name.encode()][
                    'scale'.encode()].reshape(-1)
                offset = weights[layer.name.encode()][
                    'offset'.encode()].reshape(-1)

                self.model.get_layer(layer.name).set_weights(
                    [scale, offset, mean, variance])

            elif layer.name[:4] == 'conv' and not layer.name[-4:] == 'relu':
                try:
                    weight = weights[layer.name.encode()]['weights'.encode()]
                    self.model.get_layer(layer.name).set_weights([weight])
                except Exception as err:
                    biases = weights[layer.name.encode()]['biases'.encode()]
                    self.model.get_layer(layer.name).set_weights([weight,
                                                                  biases])
        print('Finished importing weights.')

        print("Writing keras model & weights")
        json_string = self.model.to_json()
        with open(json_path, 'w') as file_handle:
            file_handle.write(json_string)
        self.model.save_weights(h5_path)
        print("Finished writing Keras model & weights")


class PSPNet50(PSPNet):
    """Build a PSPNet based on a 50-Layer ResNet."""

    def __init__(self, nb_classes, weights, input_shape):
        PSPNet.__init__(self, nb_classes=nb_classes, resnet_layers=50,
                        input_shape=input_shape, weights=weights)


class PSPNet101(PSPNet):
    """Build a PSPNet based on a 101-Layer ResNet."""

    def __init__(self, nb_classes, weights, input_shape):
        PSPNet.__init__(self, nb_classes=nb_classes, resnet_layers=101,
                        input_shape=input_shape, weights=weights)

In [3]:
def loading_bbox(img_path):
    img_nm = img_path.split('/')[-1]
    
    if img_nm in list(bbox_all.keys()):
        bbox = bbox_all[img_nm]
        pass
    
    else:
        msg = '>>> Error : No Bbox in this picture!'
        print(msg)
        raise Exception
        pass
    
    return bbox    

## ( 계정명 & 색상 추출 개수 ) 입력

In [4]:
account_name = "world_fashion_styles"
expected_num_col = 5

## Bbox( json ) 읽기

In [5]:
account_nm = account_name + '_bbox.json'

with open("../Trip_Fashion_Recommend/data/" + account_nm, 'r') as bbox_tmp:
    bbox_all = json.load(bbox_tmp)

## Segmentation

In [6]:
img_loc = glob("../Trip_Fashion_Recommend/data/pictures_{}/*".format(account_name))

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4 
sess = tf.Session(config=config)

K.set_session(sess)

model = "pspnet50_ade20k"

with sess.as_default():
    pspnet = PSPNet50(nb_classes=150, input_shape=(473, 473),
                              weights=model)
    lb_dict = {}
    begin_time = time.time()
    for i, img_path in enumerate(img_loc[:2]):
            print("Processing image {} / {}".format(i + 1, len(img_loc)))
            print("image : {}".format(img_path.split('/')[-1]))
            img = imread(img_path, pilmode="RGB")

            probs = pspnet.predict_multi_scale(img, flip_evaluation=True, sliding_evaluation = True,  scales=[1.0])

            cm = np.argmax(probs, axis=2)
            

            try:
                bbox = loading_bbox(img_path)
                
                # Segmentation 처리가 된 이미지에서 bbox에 해당하는 부분만 cropping
                xx = cm[bbox[0][1]:bbox[0][3], bbox[0][0]:bbox[0][2]]
                unique_elements, counts_elements = np.unique(xx, return_counts=True)
                unique_elements[np.where(counts_elements==max(counts_elements))][0]
                a,b = np.where(xx == unique_elements[np.where(counts_elements==max(counts_elements))][0])
                img_crop = img[bbox[0][1]:bbox[0][3], bbox[0][0]:bbox[0][2]]
#                 plt.imshow(img_crop)
#                 plt.show()

                # Extract labels & cluster_centers_ by using KMeans
                clt = KMeans(n_clusters = expected_num_col)
                clt.fit(img_crop[a,b])
                lb_counts = Counter(clt.labels_)

                tmp_dict = {}
                filename = img_path.split('/')[-1]
                for idx, pixel in enumerate(clt.cluster_centers_):
                    tmp_dict[ str(lb_counts[idx]) ] = pixel.tolist()  # { frequency of label : color of label } mapping
                    lb_dict[filename] = tmp_dict
                    pass
                pass
            
            except Exception as e:
                msg = '>>> Error in {}th file : {}\n'.format(i+1, img_path)
                print(msg)

            
            
            
            
            
    # json 파일에 lb_count , clt.cluster_centers_ 저장
    save_path = "../data/{}.json".format(account_name)
    with open(save_path, 'w') as json_file:
        json.dump(lb_dict, json_file)

end_time = time.time()
  
print('time elapsed while training : {}'.format(end_time - begin_time))

Keras model & weights found, loading...






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Processing image 1 / 80
image : world_fashion_styles_124_1299Likes.png
Started prediction...
Predicting image scaled by 1.000000
Need 3 x 4 prediction tiles @ stride 316 px
Predicting tile 1
Predict flipped


/home/ubuntu/anaconda3/envs/pspnet_env/lib/python3.5/site-packages/ipykernel_launcher.py:108: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Predicting tile 2
Predict flipped
Predicting tile 3
Predict flipped
Predicting tile 4
Predict flipped
Predicting tile 5
Predict flipped
Predicting tile 6
Predict flipped
Predicting tile 7
Predict flipped
Predicting tile 8
Predict flipped
Predicting tile 9
Predict flipped
Predicting tile 10
Predict flipped
Predicting tile 11
Predict flipped
Predicting tile 12
Predict flipped
Finished prediction...
Processing image 2 / 80
image : world_fashion_styles_002_2678Likes.png
Started prediction...
Predicting image scaled by 1.000000
Need 3 x 4 prediction tiles @ stride 316 px
Predicting tile 1
Predict flipped


/home/ubuntu/anaconda3/envs/pspnet_env/lib/python3.5/site-packages/ipykernel_launcher.py:108: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Predicting tile 2
Predict flipped
Predicting tile 3
Predict flipped
Predicting tile 4
Predict flipped
Predicting tile 5
Predict flipped
Predicting tile 6
Predict flipped
Predicting tile 7
Predict flipped
Predicting tile 8
Predict flipped
Predicting tile 9
Predict flipped
Predicting tile 10
Predict flipped
Predicting tile 11
Predict flipped
Predicting tile 12
Predict flipped
Finished prediction...
time elapsed while training : 192.39666199684143
